Docker was one of these things that I always wanted to learn, but never got into. Part of the reason was that it seemd distant and even somewhat unnecessary to me. As someone who has only worked on relatively simple projects, I never felt the need to go beyond the notion of virtual environments. Indeed, when I first read about Docker in an attempt to learn more about what all the DevOps hype was about, I found myself wondering: is Docker really that much different from a Python virtual environment? 

Well, some time has passed since then, and I got lucky enough to have landed an internship at a small startup. Given that the team will be using some DevOps tools---Docker definitely being one of them---I thought I'd get my hands dirty to get a sense of what Docker is like and what it's primarily used for. Instead of the YouTube route, this time I decided to check out a book titled [Docker Deep Dive](https://www.amazon.com/Docker-Deep-Dive-Nigel-Poulton-ebook/dp/B01LXWQUFF) by Nigel Poulton. Throughout this post, I will be referring to examples from his book. For those who want to get a beginner-friendly introduction to Docker, I highly recommend this book. 

At the point of writing, I've read up to Chapter 8 of the book, "Containerizing an App," immediately before the next chapter on Docker compose. This post is not intended as a comprehensive, well-written introduction to Docker; instead, it is in fact a playground environment I used to test out some Docker commands as I was following along the book. With that out of the way, let's jump right in.

# Terminal in Jupyter

Before getting into any details about Docker, it's perhaps necessary for me to clarify the setup in which this post was written. In testing out Docker commands, I went back and forth between this Jupyter notebook and the terminal. I mainly tried to use Jupyter in order to record the commands I typed and their outputs in this post, but certain commands that require secondary input in interactive mode, such as `docker container run -it [...]` was tested in the terminal.

The `!` sign in front of every Docker command is necessary to run unix commands in Jupyter. An exception is `%cd`, which is a magic command in Jupyter that allows the use of `cd`; `! cd` does not work, because the way Jupyter interacts with the system is by attaching a shell subprocess. These details aside, the key takeaway is that the exclaimation or percent symbols can be disregarded.

# Docker Basics

In this section, we will learn about some basic docker commands to get started. Here is the most basic one that allows us to check the version and configuration of Docker:

In [2]:
! docker version

Client: Docker Engine - Community
 Version:           19.03.8
 API version:       1.40
 Go version:        go1.12.17
 Git commit:        afacb8b
 Built:             Wed Mar 11 01:21:11 2020
 OS/Arch:           darwin/amd64
 Experimental:      false

Server: Docker Engine - Community
 Engine:
  Version:          19.03.8
  API version:      1.40 (minimum version 1.12)
  Go version:       go1.12.17
  Git commit:       afacb8b
  Built:            Wed Mar 11 01:29:16 2020
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          v1.2.13
  GitCommit:        7ad184331fa3e55e52b890ea95e65ba581ae3429
 runc:
  Version:          1.0.0-rc10
  GitCommit:        dc9208a3303feef5b3839f4323d9beb36df0a9dd
 docker-init:
  Version:          0.18.0
  GitCommit:        fec3683


Notice that the Docker engine correctly identifies as `OS/Arch` as `darwin`, whereas that of the Server is noted as `linux`. In essence, this is saying that the server is running on a linux kernel. Running a linux kernel on a macOS host through Docker is made possible via Hypervisor and the LinuxKit. At this point, all there is to know about the details is that Docker originally used VirtualBox to run a linux VM, but now uses a more lightweight setup thanks to the aforementioned tools. 

# ls Commands

In unix, `ls` is a command that can be used to get a convenient list of files available in the current directory. Similarly, `docker [...] ls` can be used to look up what docker components are running or existent. For instance, to check which containers are running, we can type

In [1]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


If we want to check images instead of containers, we can simply replace the `container` with `image`.

In [5]:
! docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
test                latest              3ad97d9a5a5a        13 minutes ago      82.7MB
alpine              latest              a24bb4013296        6 weeks ago         5.57MB
golang              1.11-alpine         e116d2efa2ab        10 months ago       312MB


# Pulling an Image

To pull an image, we can use `docker pull [...]`, where the elipses are the name of the repository and the tag. For examplle, 

In [6]:
! docker pull ubuntu:latest

latest: Pulling from library/ubuntu

352adcf2: Pulling fs layer 
8a342707: Pulling fs layer 
b8e766f4: Pulling fs layer 
Digest: sha256:55cd38b70425947db71112eb5dddfa3aa3e3ce307754a3df2269069d2278ce47[4A
Status: Downloaded newer image for ubuntu:latest
docker.io/library/ubuntu:latest


In [7]:
! docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
test                latest              3ad97d9a5a5a        13 minutes ago      82.7MB
ubuntu              latest              adafef2e596e        6 days ago          73.9MB
alpine              latest              a24bb4013296        6 weeks ago         5.57MB
golang              1.11-alpine         e116d2efa2ab        10 months ago       312MB


In [12]:
! docker container run -it ubuntu:latest /bin/bash

In [14]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
6258444a446a        ubuntu:latest       "/bin/bash"         35 seconds ago      Up 34 seconds                           compassionate_hofstadter


In [15]:
! docker container stop compassionate_hofstadter

compassionate_hofstadter


In [20]:
! docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS                      PORTS               NAMES
6258444a446a        ubuntu:latest       "/bin/bash"         About a minute ago   Exited (0) 45 seconds ago                       compassionate_hofstadter


In [21]:
! docker container rm compassionate_hofstadter

compassionate_hofstadter


In [22]:
! docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [23]:
! docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
test                latest              3ad97d9a5a5a        18 minutes ago      82.7MB
ubuntu              latest              adafef2e596e        6 days ago          73.9MB
alpine              latest              a24bb4013296        6 weeks ago         5.57MB
golang              1.11-alpine         e116d2efa2ab        10 months ago       312MB


In [37]:
%cd psweb

/Users/jaketae/Documents/GitHub/psweb


In [38]:
! cat Dockerfile

# Test web-app to use with Pluralsight courses and Docker Deep Dive book
# Linux x64
FROM alpine

LABEL maintainer="nigelpoulton@hotmail.com"

# Install Node and NPM
RUN apk add --update nodejs nodejs-npm

# Copy app to /src
COPY . /src

WORKDIR /src

# Install dependencies
RUN  npm install

EXPOSE 8080

ENTRYPOINT ["node", "./app.js"]


In [39]:
! docker image build -t test:latest .

Sending build context to Docker daemon  100.9kB
Step 1/8 : FROM alpine
 ---> a24bb4013296
Step 2/8 : LABEL maintainer="nigelpoulton@hotmail.com"
 ---> Using cache
 ---> 2ead764f71cf
Step 3/8 : RUN apk add --update nodejs nodejs-npm
 ---> Using cache
 ---> 6a652e727789
Step 4/8 : COPY . /src
 ---> Using cache
 ---> 33eed66ed95e
Step 5/8 : WORKDIR /src
 ---> Using cache
 ---> e07f22f7a87b
Step 6/8 : RUN  npm install
 ---> Using cache
 ---> 57fcc62715f2
Step 7/8 : EXPOSE 8080
 ---> Using cache
 ---> 889b9b226806
Step 8/8 : ENTRYPOINT ["node", "./app.js"]
 ---> Using cache
 ---> 3ad97d9a5a5a
Successfully built 3ad97d9a5a5a
Successfully tagged test:latest


In [41]:
! docker container run -d --name web1 --publish 8080:8080 test:latest

c6645ae79b55b87650c8468d1f605e34d3c22a948a2c99bf717f25753598f63a


In [42]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS                    NAMES
c6645ae79b55        test:latest         "node ./app.js"     19 seconds ago      Up 18 seconds       0.0.0.0:8080->8080/tcp   web1


In [43]:
! docker container stop c6645ae79b55

c6645ae79b55


In [45]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [46]:
! docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS                        PORTS               NAMES
c6645ae79b55        test:latest         "node ./app.js"     About a minute ago   Exited (137) 51 seconds ago                       web1


In [47]:
! docker container rm web1

web1


In [48]:
! docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [49]:
! docker container run --name ctr1 -it alpine:latest sh

/ # ^C
/ # 
/ # 

In [50]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
6295af1857c5        alpine:latest       "sh"                12 seconds ago      Up 12 seconds                           ctr1


In [51]:
! docker image rm alpine:latest

Error response from daemon: conflict: unable to remove repository reference "alpine:latest" (must force) - container 6295af1857c5 is using its referenced image a24bb4013296


```
! docker image pull ubuntu:latest
! docker image pull redis:latest
! docker image pull mongo:3.3.11
! docker image pull nigelpoulton/tu-demo:v2
```

In [54]:
! docker image ls --filter dangling=true

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [55]:
! docker image prune

WARNING! This will remove all dangling images.
Are you sure you want to continue? [y/N] ^C


In [57]:
! docker image ls --filter=reference="*:latest"

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
test                latest              3ad97d9a5a5a        2 hours ago         82.7MB
ubuntu              latest              adafef2e596e        6 days ago          73.9MB
alpine              latest              a24bb4013296        6 weeks ago         5.57MB


In [67]:
! docker search nigelpoulton | head

NAME                                 DESCRIPTION                                     STARS               OFFICIAL            AUTOMATED
nigelpoulton/pluralsight-docker-ci   Simple web app used in my Pluralsight video …   23                                      [OK]
nigelpoulton/tu-demo                 Voting web server used for various Pluralsig…   12                                      
nigelpoulton/ctr-demo                Web server for simple Docker demos              3                                       
nigelpoulton/k8sbook                 Simple web app used for demos in The Kuberne…   2                                       
nigelpoulton/vote                    Fork of dockersamples Voting App for *Docker…   1                                       
nigelpoulton/dockerbook              Repo for examples used in Docker Deep Dive b…   0                                       
nigelpoulton/msb-hello                                                               0                   

In [68]:
! docker search alpine --filter "is-official=true"

NAME                DESCRIPTION                                     STARS               OFFICIAL            AUTOMATED
alpine              A minimal Docker image based on Alpine Linux…   6613                [OK]                


In [69]:
! docker image inspect ubuntu:latest

[
    {
        "Id": "sha256:adafef2e596ef06ec2112bc5a9663c6a4f59a3dfd4243c9cabe06c8748e7f288",
        "RepoTags": [
            "ubuntu:latest"
        ],
        "RepoDigests": [
            "ubuntu@sha256:55cd38b70425947db71112eb5dddfa3aa3e3ce307754a3df2269069d2278ce47"
        ],
        "Parent": "",
        "Comment": "",
        "Created": "2020-07-06T21:56:31.471255509Z",
        "Container": "6255a9da773a5e0438e3c097b876a2de65d33f3fb57c4e515faed215d17b8b5d",
        "ContainerConfig": {
            "Hostname": "6255a9da773a",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
            ],
            "Cmd": [
                "/bin/sh",
                "-c",
                "#(

In [71]:
! docker image pull -a nigelpoulton/tu-demo

latest: Pulling from nigelpoulton/tu-demo

3a933944: Pulling fs layer 
563217f5: Pulling fs layer 
7ec39263: Pulling fs layer 
26f0f7cc: Pulling fs layer 
2aee5115: Pulling fs layer 
e9939cc3: Pulling fs layer 
38d27074: Pulling fs layer 
8469a194: Pulling fs layer 
Digest: sha256:c9f8e1882275d9ccd82e9e067c965d1406e8e1307333020a07915d6cbb9a74cf[7A
v1: Pulling from nigelpoulton/tu-demo

3a933944: Already exists 
563217f5: Already exists 
7ec39263: Already exists 
26f0f7cc: Already exists 
2aee5115: Already exists 
e9939cc3: Already exists 
38d27074: Already exists 
8469a194: Already exists 
Digest: sha256:674cb034447ab34d442b8df03e0db6506a99390a1e282d126fb44af8598e4d2a
v2: Pulling from nigelpoulton/tu-demo
Digest: sha256:c9f8e1882275d9ccd82e9e067c965d1406e8e1307333020a07915d6cbb9a74cf
Status: Downloaded newer image for nigelpoulton/tu-demo
docker.io/nigelpoulton/tu-demo


In [87]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [88]:
! docker container run alpine:latest sleep 10

In [89]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [91]:
! systemctl is-active docker

/bin/sh: systemctl: command not found


In [93]:
! docker stop

"docker stop" requires at least 1 argument.
See 'docker stop --help'.

Usage:  docker stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers


always restart unless stopped explicitly
```
! docker container run --name neversaydie -it --restart always alpine sh
! docker container run --name neversaydie -it --restart unless-stopped alpine sh
! docker container run --name neversaydie -it --restart on-failure alpine sh
```

In Docker container: 8080
On actual host (laptop): 80
```
! docker container run -d --name webserver -p 80:8080 nigelpoulton/pluralsight-docker-ci
```

In [101]:
! docker image inspect nigelpoulton/pluralsight-docker-ci

[
    {
        "Id": "sha256:dd7a37fe7c1e6f3b9bcd1c51cad0a54fde3f393ac458af3b009b2032978f599d",
        "RepoTags": [
            "nigelpoulton/pluralsight-docker-ci:latest"
        ],
        "RepoDigests": [
            "nigelpoulton/pluralsight-docker-ci@sha256:61bc64850a5f2bfbc65967cc33feaae8a77c8b49379c55aaf05bb02dcee41451"
        ],
        "Parent": "",
        "Comment": "",
        "Created": "2020-01-18T15:29:24.3067368Z",
        "Container": "5e6c8e135f3504d8cdbb3b0f4f7658018f7eafa99011bcb0252c34bad246844f",
        "ContainerConfig": {
            "Hostname": "5e6c8e135f35",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "ExposedPorts": {
                "8080/tcp": {}
            },
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/usr/local/sbin:/usr/local/bin:/usr/

```
"Cmd": [
                "/bin/sh",
                "-c",
                "#(nop) ",
                "CMD [\"/bin/sh\" \"-c\" \"cd /src && node ./app.js\"]"
```
Translates to
```
/bin/sh -c "cd /src && node ./app.js"
```

In [106]:
! docker container run alpine:latest sh

In [107]:
! docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                     PORTS               NAMES
da65774cecf9        alpine:latest       "sh"                9 seconds ago       Exited (0) 8 seconds ago                       gallant_babbage


In [108]:
! docker container rm $(docker container ls -aq) -f

da65774cecf9


In [143]:
! cat Dockerfile

# Test web-app to use with Pluralsight courses and Docker Deep Dive book
# Linux x64
FROM alpine

LABEL maintainer="nigelpoulton@hotmail.com"

# Install Node and NPM
RUN apk add --update nodejs nodejs-npm

# Copy app to /src
COPY . /src

WORKDIR /src

# Install dependencies
RUN  npm install

EXPOSE 8080

ENTRYPOINT ["node", "./app.js"]


```
layer 4: RUN  npm install
========================================
layer 3: COPY . /src
========================================
layer 2: RUN apk add --update nodejs nodejs-npm
========================================
layer 1: FROM alpine
```

In [144]:
! docker image build -t web:latest .

Sending build context to Docker daemon    105kB
Step 1/8 : FROM alpine
 ---> a24bb4013296
Step 2/8 : LABEL maintainer="nigelpoulton@hotmail.com"
 ---> Running in 21816bd078a0
Removing intermediate container 21816bd078a0
 ---> df3b1c80ebed
Step 3/8 : RUN apk add --update nodejs nodejs-npm
 ---> Running in c0e587d4b1dd
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz
(1/9) Installing ca-certificates (20191127-r4)
(2/9) Installing brotli-libs (1.0.7-r5)
(3/9) Installing c-ares (1.16.1-r0)
(4/9) Installing libgcc (9.3.0-r2)
(5/9) Installing nghttp2-libs (1.41.0-r0)
(6/9) Installing libstdc++ (9.3.0-r2)
(7/9) Installing libuv (1.37.0-r0)
(8/9) Installing nodejs (12.17.0-r0)
(9/9) Installing npm (12.17.0-r0)
Executing busybox-1.31.1-r16.trigger
Executing ca-certificates-20191127-r4.trigger
OK: 65 MiB in 23 packages
Removing intermediate container c0e587d4b1dd
 ---> f139ee287703
Step 

In [145]:
! docker image ls

REPOSITORY                           TAG                 IMAGE ID            CREATED             SIZE
web                                  latest              34b07893e6cf        10 seconds ago      82.8MB
ubuntu                               latest              adafef2e596e        6 days ago          73.9MB
alpine                               latest              a24bb4013296        6 weeks ago         5.57MB
nigelpoulton/pluralsight-docker-ci   latest              dd7a37fe7c1e        5 months ago        604MB
golang                               1.11-alpine         e116d2efa2ab        10 months ago       312MB


In [147]:
! docker image tag web:latest jaketae/web:latest

In [148]:
! docker image ls

REPOSITORY                           TAG                 IMAGE ID            CREATED             SIZE
jaketae/web                          latest              34b07893e6cf        30 minutes ago      82.8MB
web                                  latest              34b07893e6cf        30 minutes ago      82.8MB
ubuntu                               latest              adafef2e596e        6 days ago          73.9MB
alpine                               latest              a24bb4013296        6 weeks ago         5.57MB
nigelpoulton/pluralsight-docker-ci   latest              dd7a37fe7c1e        5 months ago        604MB
golang                               1.11-alpine         e116d2efa2ab        10 months ago       312MB


In [149]:
! docker image push jaketae/web:latest

The push refers to repository [docker.io/jaketae/web]

8b6e0356: Preparing 
9a0747a8: Preparing 
a1bd40b4: Preparing 
a1bd40b4: Pushed   54.46MB/51MB5MBine latest: digest: sha256:ffac23f83cc6f8e6a888db08dc95eca411b13548db499be994f24c26826ac532 size: 1161


In [152]:
! docker container run -d --name c1 -p 80:8080 web:latest

8b867dd4a2843e6fb889c66da86c3b03a9fb3cbe2c1190a110826b6063881204


In [153]:
! docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS                  NAMES
8b867dd4a284        web:latest          "node ./app.js"     5 seconds ago       Up 4 seconds        0.0.0.0:80->8080/tcp   c1


In [155]:
! docker container stop 8b867dd4a284; docker container rm 8b867dd4a284

8b867dd4a284
8b867dd4a284


In [156]:
! docker image history web:latest

IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
34b07893e6cf        36 minutes ago      /bin/sh -c #(nop)  ENTRYPOINT ["node" "./app…   0B                  
cc0de787a103        36 minutes ago      /bin/sh -c #(nop)  EXPOSE 8080                  0B                  
13ca3e80464c        36 minutes ago      /bin/sh -c npm install                          26.1MB              
7bea6d04eb9d        36 minutes ago      /bin/sh -c #(nop) WORKDIR /src                  0B                  
83a82475aaea        36 minutes ago      /bin/sh -c #(nop) COPY dir:c206ee42d7bc6b296…   32.6kB              
f139ee287703        36 minutes ago      /bin/sh -c apk add --update nodejs nodejs-npm   51MB                
df3b1c80ebed        36 minutes ago      /bin/sh -c #(nop)  LABEL maintainer=nigelpou…   0B                  
a24bb4013296        6 weeks ago         /bin/sh -c #(nop)  CMD ["/bin/sh"]              0B                  
<missing>   

Non-zero `SIZE` corresponds to Dockerfile instruction that creates layers.